In [ ]:
# imports
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [ ]:
!ollama pull llama3.2

# instantiating variables
HEADERS = {"Content-Type": "application/json"}
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# a method for displaying a message in markdown form
def display_in_markdown(message):
    return display(Markdown(message))

# a class for crawling a website using beautiful soup
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_text(self):
        return f"{self.title}\n{self.text}"

In [ ]:
# method for summarize a website using beautifulsoup and deepseek
def summarize_website(website):
    # creates the user prompt
    website_text = Website(website).get_text()
    user_prompt = f"You are looking at a website titled \"{Website(website).title}\""
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these, ignoring text that might be navigation related too.\n\n"
    user_prompt += website_text

    # fetches the response from ollama
    response = ollama.chat(model="llama3.2", messages=[{"role": "user", "content": user_prompt}])
    return response.get("message", {}).get("content", "No response received.")

In [ ]:
print(display_in_markdown(summarize_website("https://wchiming.com/blog/bitcoin")))